## Dataset
In this homework, we'll build a model for classifying various hair types. For this, we will use the Hair Type dataset that was obtained from [Kaggle](https://www.kaggle.com/datasets/kavyasreeb/hair-type-dataset) and slightly rebuilt.

You can download the target dataset for this homework from [here](https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip):
```bash
wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
unzip data.zip
```
In the lectures we saw how to use a pre-trained neural network. In the homework, we'll train a much smaller model from scratch.

>Note: you will need an environment with a GPU for this homework. We recommend to use [Saturn Cloud](https://bit.ly/saturn-mlzoomcamp). You can also use a computer without a GPU (e.g. your laptop), but it will be slower.

## Data Preparation
The dataset contains around 1000 images of hairs in the separate folders for training and test sets.

## Reproducibility
Reproducibility in deep learning is a multifaceted challenge that requires attention to both software and hardware details. In some cases, we can't guarantee exactly the same results during the same experiment runs. Therefore, in this homework we suggest to:

* install tensorflow version 2.17.1
* set the seed generators by:
```python
import numpy as np
import tensorflow as tf

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
```

In [ ]:
!pip install tensorflow==2.17.1

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

train_dir = './data/train'
test_dir = './data/test'

## Model
For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

* The shape for input should be `(200, 200, 3)`.
* Next, create a convolutional layer ([Conv2D](https://keras.io/api/layers/convolution_layers/convolution2d/)):
    * Use 32 filters.
    * Kernel size should be `(3, 3)` (that's the size of the filter).
    * Use `'relu'` as activation.
* Reduce the size of the feature map with max pooling ([MaxPooling2D](https://keras.io/api/layers/pooling_layers/max_pooling2d/)).
* Set the pooling size to `(2, 2)`.
* Turn the multi-dimensional result into vectors using a [Flatten](https://keras.io/api/layers/reshaping_layers/flatten/) layer
* Next, add a [Dense](https://keras.io/api/layers/core_layers/dense/) layer with 64 neurons and `'relu'` activation.
* Finally, create the `Dense` layer with 1 neuron - this will be the output.
    * The output layer should have an activation - use the appropriate activation for the binary classification case
As optimizer use [SGD](https://keras.io/api/optimizers/sgd/) with the following parameters:
    * `SGD(lr=0.002, momentum=0.8)`
For clarification about kernel size and max pooling, check [Office Hours](https://www.youtube.com/watch?v=1WRgdBTUaAc).

In [ ]:
inputs = tf.keras.Input(shape=(200,200,3))
Conv2d = tf.keras.layers.Conv2D(filters=32, 
                                kernel_size=(3,3),
                                activation='relu')(inputs)
pooling = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(Conv2d)
flatten = tf.keras.layers.Flatten()(pooling)
dense = tf.keras.layers.Dense(64, activation='relu')(flatten)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
lr = 0.002
momentum = 0.8
optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=momentum)

loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

## Question 1
Since we have a binary classification problem, what is the best loss function for us?

* `mean squared error`
* `binary crossentropy`
* `categorical crossentropy`
* `cosine similarity`

Note: since we specify an activation for the output layer, we don't need to set `from_logits=True`

**Answer**: binary crossentropy

## Question 2
What's the total number of parameters of the model? You can use the `summary` method for that.

* 896
* 11214912
* 15896912
* 20072512

In [ ]:
model.summary()

## Generators and Training
For the next two questions, use the following data generator for both train and test sets:

[`ImageDataGenerator(rescale=1./255)`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)
* We don't need to do any additional pre-processing for the images.
* When reading the data from train/test directories, check the `class_mode` parameter. Which value should it be for a binary classification problem?
* Use `batch_size=20`
* Use `shuffle=True` for both training and test sets.

For training use [.fit()](https://keras.io/api/models/model_training_apis/#fit-method) with the following params:
```python
model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)
```

In [ ]:
dir_params = {class_mode='binary',
              batch_size=20,
              shuffle=True
            }

In [ ]:
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
                                                                        train_dir,
                                                                        **dir_params
                                                                    )

In [ ]:
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
                                                                        test_dir,
                                                                        **dir_params
                                                                    )

In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

In [ ]:
history

## Question 3
What is the median of training accuracy for all the epochs for this model?

* 0.10
* 0.32
* 0.50
* 0.72

## Question 4
What is the standard deviation of training loss for all the epochs for this model?

* 0.028
* 0.068
* 0.128
* 0.168

## Data Augmentation
For the next two questions, we'll generate more data using data augmentations. See [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator).

Add the following augmentations to your training data generator:

* `rotation_range=50`,
* `width_shift_range=0.1`,
* `height_shift_range=0.1`,
* `zoom_range=0.1`,
* `horizontal_flip=True`,
* `fill_mode='nearest'`

In [ ]:
train_gen = ImageDataGenerator(rescale=1./255,
                            rotation_range=50,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.1,
                            horizontal_flip=True,
                            fill_mode='nearest'
                            )

In [ ]:
train_generator = train_gen.flow_from_directory(
                                                train_dir,
                                                **dir_params
                                            )

## Question 5
Let's train our model for 10 more epochs using the same code as previously.

>Note: make sure you don't re-create the model - we want to continue training the model we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

* 0.26
* 0.56
* 0.86
* 1.16

In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

In [ ]:
history

## Question 6
What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

* 0.31
* 0.51
* 0.71
* 0.91